In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from networktables import NetworkTables
import skimage.color

NetworkTables.initialize(server='10.xx.xx.2')
table = NetworkTablesInstance.getTable('SmartDashboard')

PIXEL_BUFFER = 75

#Dimensions of RR tape
REAL_HEIGHT = 5 #cm
REAL_WIDTH = 13 #cm

#RR tape masks
hsvLow = np.array([40, 40, 40])
hsvHigh = np.array([80, 255, 255])

#Goal detection yuv thresholds
meanR = [0.64853304, 0.02986617, -0.33895488]
stddevR = [0.06069028, 0.02728164, 0.06637001]

#Limelight constants
PIXELS_WIDE = 320 #pixels
PIXELS_HIGH = 240 #pixels
FOV_Y = 49.7 #degrees
FOV_X = 59.6 #degrees
FOCAL_LENGTH = (PIXELS_HIGH / 2) / np.tan(math.pi / 180 * FOV_Y / 2)

#Other parameters
TARGET_HEIGHT = 260 #cm (measured from the top of the target to the ground)
CAMERA_HEIGHT = 28 / 12 * 30 #cm
CAMERA_ANGLE = 27 # degrees

def align_with_hub(centerX):
  if centerX < PIXELS_WIDE/2 + PIXEL_BUFFER and centerX > PIXELS_WIDE/2 - PIXEL_BUFFER:
    print("ready to shoot")
    print(centerX)
    table.putNumber("alignment", 0)

  elif centerX > PIXELS_WIDE/2 - PIXEL_BUFFER:
    # move slightly right
    print("move right")
    print(centerX)
    table.putNumber("alignment", 1)
    #align_with_hub() # put new photo after moving robot

  else:
    # move slightly left
    print("move left")
    print(centerX)
    table.putNumber("alignment", -1)
    #align_with_hub() # put new photo after moving robot

def draw_contour(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  img = img[int(PIXELS_HIGH/5):int(3*PIXELS_HIGH/5),:]
  img = cv2.blur(img, (2, 2))

  imgray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

  #ret -> threshold value    thresh -> thresholded image
  ret, thresh = cv2.threshold(imgray, 10, 240, 0) 

  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  liz = [] #list of contours with correct aspect ratio
  leftX = 400
  rightX = -1
  topY = -1
  for contour in contours:
    x,y,width,height = cv2.boundingRect(contour)
    aspect = width/height #should be around 2.5 (5/2)
    if(aspect < 5 and aspect > 0.5 and height < width and width*height < 600 and width*height > 30):
      liz.append(contour)
      if x < leftX: 
        leftX = x
      if x + width > rightX:
        rightX = x + width
      topY = y

  img = skimage.color.yuv2rgb(img)

  for con in liz:
    x, y, w, h = cv2.boundingRect(con)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
    print(w, h)

  #returns center x-value of the contours
  return (rightX + leftX)/2, topY

#returns the horizontal distance (cm) from the RR tape to the camera
def getDistance(y):  
  #normalizing the top y position of the goal contour to be from [-1,1]
  norm_Y = - (y - (PIXELS_HIGH / 2)) / (PIXELS_HIGH / 2)

  #finding the vertical angle (pitch) between the camera and the goal contour 
  pitch = 180 * np.arctan(norm_Y * np.tan(FOV_Y / 2 * math.pi / 180)) / math.pi # degrees
  
  #using trig to determine distance
  distance = (TARGET_HEIGHT - CAMERA_HEIGHT) / np.tan(math.pi * (CAMERA_ANGLE + pitch) / 180)
  table.putNumber("distance", distance)
  print(distance)

  #return ([[[0, 0]], [[0, 100]], [[100, 0]], [[100, 100]]], cv2.drawContours(output_image, [goal], 0, (0, 0, 255), 3), output)

def runPipeline(image, llrobot):
  im = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  imgyuv = skimage.color.rgb2yuv(image)

  #meanR = [0.64853304, 0.02986617, -0.33895488]
  #stddevR = [0.06069028, 0.02728164, 0.06637001]

  scale = 6
  low_threshold = np.array([0.0,
                          meanR[1] - scale * stddevR[1],
                          meanR[2] - scale * stddevR[2]])

  high_threshold = np.array([1.0,
                          meanR[1] + scale * stddevR[1],
                          meanR[2] + scale * stddevR[2]])

  background_mask = np.all((imgyuv > low_threshold) * (imgyuv < high_threshold), axis=2)
  background_mask = np.where(background_mask, 1, 0)
  background_mask = np.expand_dims(background_mask, 2)

  im = im*background_mask
  align, y = draw_contour(im)
  align_with_hub(align)
  getDistance(y)

In [ ]:
#Pre-game calibration routine
def calibrate(knownimage):
  test = cv2.imread(knownimage) 

  testyuv = skimage.color.rgb2yuv(test)

  x1, x2 = 0
  y1, y2 = 0
  knownyuv = skimage.color.rgb2yuv(th[y1:y2, x1:x2])

  knownyuv = np.reshape(knownyuv, (-1, 3))

  mean_yuv = knownyuv.mean(axis = 0)
  stddev_yuv = knownyuv.std(axis = 0)

  meanR = mean_yuv
  stddevR = stddev_yuv

  test_calibrate(meanR, stddevR)

def test_calibrate(imgyuv, meanR, stddevR):
  scale = 4
  low_threshold = np.array([0.0,
                            meanR[1] - scale * stddevR[1],
                            meanR[2] - scale * stddevR[2]])

  high_threshold = np.array([1.0,
                            meanR[1] + scale * stddevR[1],
                            meanR[2] + scale * stddevR[2]])

  background_mask = np.all((imgyuv > low_threshold) * (imgyuv < high_threshold), axis=2)
  background_mask = np.where(background_mask, 1, 0)
  background_mask = np.expand_dims(background_mask, 2)

  th = th*background_mask
  plt.imshow(th)
  plt.show()